In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd

import folium
from folium import plugins
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster
from geopy.geocoders import Nominatim


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('../data/df_imputed.csv')
df.head()

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,...,neighbourhood_cleansed_Vila Isabel,neighbourhood_cleansed_Vila Kosmos,neighbourhood_cleansed_Vila Militar,neighbourhood_cleansed_Vila Valqueire,neighbourhood_cleansed_Vila da Penha,neighbourhood_cleansed_Zumbi,neighbourhood_cleansed_Água Santa,property_category_Other,property_category_Private,property_category_Shared
0,783493769216852616,100.0,67.0,0,2.0,2.0,0,0,-22.96421,-43.171600,...,0,0,0,0,0,0,0,0,0,0
1,703973293620197060,70.0,100.0,0,4.0,5.0,1,1,-22.95792,-43.182226,...,0,0,0,0,0,0,0,0,1,0
2,782895997622988215,100.0,50.0,0,1.0,1.0,1,1,-22.93196,-43.180180,...,0,0,0,0,0,0,0,0,0,0
3,23768085,100.0,100.0,0,1.0,2.0,1,1,-23.01104,-43.320340,...,0,0,0,0,0,0,0,0,0,0
4,10463735,78.0,100.0,0,6.0,6.0,1,0,-22.93555,-43.191070,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.shape

(28291, 197)

In [11]:
print(df.corr()['price'].sort_values(ascending = False))

price                                   1.000000
baths                                   0.084337
accommodates                            0.081013
bedrooms                                0.078485
neighbourhood_cleansed_São Cristóvão    0.058149
                                          ...   
review_scores_rating                   -0.019319
latitude                               -0.021043
number_of_reviews_ltm                  -0.021734
reviews_per_month                      -0.022454
property_category_Private              -0.035454
Name: price, Length: 197, dtype: float64


In [12]:
df2 = pd.read_csv('../data/data.csv')
df2.head()

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,100.0,67.0,0,2.0,2.0,0,0,Leme,-22.96421,...,NaN,NaN,0,2,2,0,0,NaN,1.0,Entire
1,703973293620197060,70.0,100.0,0,4.0,5.0,1,1,Botafogo,-22.95792,...,4.00,5.00,1,4,0,4,0,0.26,1.0,Private
2,782895997622988215,100.0,50.0,0,1.0,1.0,1,1,Laranjeiras,-22.93196,...,NaN,NaN,0,1,1,0,0,NaN,1.0,Entire
3,23768085,100.0,100.0,0,1.0,2.0,1,1,Barra da Tijuca,-23.01104,...,4.98,4.78,1,1,1,0,0,1.59,1.0,Entire
4,10463735,78.0,100.0,0,6.0,6.0,1,0,Laranjeiras,-22.93555,...,2.00,1.00,1,6,2,4,0,0.03,2.0,Entire


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28291 entries, 0 to 28290
Data columns (total 45 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            28291 non-null  int64  
 1   host_response_rate                            23888 non-null  float64
 2   host_acceptance_rate                          24542 non-null  float64
 3   host_is_superhost                             28291 non-null  int64  
 4   host_listings_count                           28291 non-null  float64
 5   host_total_listings_count                     28291 non-null  float64
 6   host_has_profile_pic                          28291 non-null  int64  
 7   host_identity_verified                        28291 non-null  int64  
 8   neighbourhood_cleansed                        28291 non-null  object 
 9   latitude                                      28291 non-null 

In [14]:
df2.corr().price.sort_values(ascending = False)

price                                           1.000000
baths                                           0.084337
accommodates                                    0.081013
bedrooms                                        0.077233
beds                                            0.055165
calculated_host_listings_count_entire_homes     0.041857
host_total_listings_count                       0.041452
calculated_host_listings_count                  0.040977
host_listings_count                             0.029835
minimum_nights_avg_ntm                          0.023040
minimum_minimum_nights                          0.022917
minimum_nights                                  0.022545
maximum_minimum_nights                          0.019189
availability_30                                 0.015143
availability_60                                 0.013784
availability_90                                 0.013722
availability_365                                0.012427
id                             

In [23]:
X = df2[['baths', 'accommodates', 'minimum_nights_avg_ntm']]
y = df2['price']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [25]:
print(f' Training Accuracy Score: {lr.score(X_train, y_train)}')
print(f' Test Accurancy Score: {lr.score(X_test, y_test)}')

 Training Accuracy Score: 0.008522334573446866
 Test Accurancy Score: 0.010793555988804493
